In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag", 'sec_app_earliest_cr_line',
    'last_credit_pull_d', 'earliest_cr_line'
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = ('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,sec_app_earliest_cr_line,last_credit_pull_d,earliest_cr_line
0,18000.0,0.1557,433.63,MORTGAGE,60000.0,Verified,Mar-2019,low_risk,n,62.80,...,0.0,196496.0,170509.0,10200.0,73796.0,N,N,Aug-2004,Apr-2019,Nov-2014
1,40000.0,0.2250,1537.99,MORTGAGE,54000.0,Verified,Mar-2019,low_risk,n,59.09,...,0.0,123858.0,107194.0,19100.0,64858.0,N,N,Jan-2003,Apr-2019,Jan-2003
2,13975.0,0.1640,342.83,RENT,67000.0,Verified,Mar-2019,low_risk,n,13.04,...,0.0,545200.0,12322.0,9600.0,9200.0,N,N,Jan-2012,Apr-2019,Jul-2004
3,40000.0,0.1171,883.93,MORTGAGE,79000.0,Verified,Mar-2019,low_risk,n,21.89,...,0.0,520784.0,111899.0,76700.0,71224.0,N,N,Nov-2006,Apr-2019,Aug-2005
4,35000.0,0.0702,1081.02,OWN,50000.0,Not Verified,Mar-2019,low_risk,n,38.93,...,0.0,135831.0,68399.0,32100.0,85531.0,N,N,Jun-2005,Apr-2019,Nov-2003


In [6]:
#Create Datafram

df_training_variables = pd.get_dummies(df, columns=[
    'initial_list_status',
    'sec_app_earliest_cr_line',
    'home_ownership',
    'verification_status',
    'issue_d',
    'pymnt_plan',
    'application_type',
    'hardship_flag',
    'last_credit_pull_d',
    'next_pymnt_d',
    'earliest_cr_line',
    'debt_settlement_flag'])
df_training_variables

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,earliest_cr_line_Sep-2007,earliest_cr_line_Sep-2008,earliest_cr_line_Sep-2009,earliest_cr_line_Sep-2010,earliest_cr_line_Sep-2011,earliest_cr_line_Sep-2012,earliest_cr_line_Sep-2013,earliest_cr_line_Sep-2014,earliest_cr_line_Sep-2015,debt_settlement_flag_N
0,18000.0,0.1557,433.63,60000.0,low_risk,62.80,0.0,0.0,9.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,40000.0,0.2250,1537.99,54000.0,low_risk,59.09,0.0,0.0,19.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,13975.0,0.1640,342.83,67000.0,low_risk,13.04,0.0,0.0,6.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,40000.0,0.1171,883.93,79000.0,low_risk,21.89,1.0,0.0,29.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,35000.0,0.0702,1081.02,50000.0,low_risk,38.93,0.0,0.0,14.0,0.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9606,40000.0,0.1894,1465.03,55000.0,low_risk,18.08,0.0,1.0,11.0,0.0,...,0,0,0,0,0,0,0,0,0,1
9607,40000.0,0.2235,1112.74,100000.0,low_risk,46.31,4.0,0.0,13.0,0.0,...,0,0,0,0,0,0,0,0,0,1
9608,40000.0,0.1033,1296.90,100000.0,low_risk,28.67,0.0,0.0,15.0,1.0,...,0,0,0,0,0,0,0,0,0,1
9609,20000.0,0.1502,476.01,54000.0,low_risk,48.16,0.0,1.0,13.0,0.0,...,0,0,0,0,0,0,0,0,0,1


# Split the Data into Training and Testing

In [7]:
# Create our features
X = df_training_variables.drop(columns="loan_status", axis=1)

# Create our target
y = df["loan_status"]

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,earliest_cr_line_Sep-2007,earliest_cr_line_Sep-2008,earliest_cr_line_Sep-2009,earliest_cr_line_Sep-2010,earliest_cr_line_Sep-2011,earliest_cr_line_Sep-2012,earliest_cr_line_Sep-2013,earliest_cr_line_Sep-2014,earliest_cr_line_Sep-2015,debt_settlement_flag_N
count,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,...,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.0
mean,21224.253460,0.134981,598.603060,71127.468760,35.034184,0.207054,0.431069,12.304547,0.121736,18820.379669,...,0.006867,0.004786,0.003225,0.002705,0.003121,0.002393,0.002081,0.001977,0.001145,1.0
std,11121.417441,0.049900,321.040161,50783.809018,48.054925,0.676033,0.710489,5.796924,0.329533,22504.004634,...,0.082587,0.069020,0.056705,0.051944,0.055785,0.048863,0.045572,0.044421,0.033813,0.0
min,1000.000000,0.064600,31.430000,40.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,12000.000000,0.103300,342.485000,40500.000000,18.310000,0.000000,0.000000,8.000000,0.000000,6260.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,20000.000000,0.129800,545.830000,60000.000000,27.460000,0.000000,0.000000,11.000000,0.000000,12950.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,30000.000000,0.164000,812.070000,87000.000000,40.200000,0.000000,1.000000,15.000000,0.000000,23842.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
max,40000.000000,0.301700,1676.230000,950000.000000,999.000000,14.000000,5.000000,69.000000,2.000000,321859.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     9558
high_risk      53
Name: loan_status, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(7208, 1164)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier


brf_model = BalancedRandomForestClassifier(bootstrap=True, class_weight=None,
                criterion='gini', max_depth=None, max_features='auto',
                max_leaf_nodes=None, min_impurity_decrease=0.0,
                min_samples_leaf=2, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                oob_score=False, random_state=1, replacement=False,
                sampling_strategy='auto', verbose=0, warm_start=False)
brf_model.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(min_samples_leaf=2, n_jobs=1, random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score 

predictions = brf_model.predict(X_test_scaled)

balanced_accuracy_score(y_test, predictions)

0.5866067697450899

In [27]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    matrix, index=["Actual High-Risk", "Actual Low-Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm_df

,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,3,7
Actual Low-Risk,444,1949


In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.30      0.81      0.01      0.49      0.23        10
   low_risk       1.00      0.81      0.30      0.90      0.49      0.26      2393

avg / total       0.99      0.81      0.30      0.89      0.49      0.26      2403



In [17]:
# List the features sorted in descending order by feature importance
pd.set_option("display.max_rows", None, "display.max_columns", None)

importances = pd.DataFrame(brf_model.feature_importances_, index = X_train.columns, columns=['Importance']).sort_values('Importance', ascending=False)
importances.loc[:,:]

,Importance
total_rec_prncp,0.049589
last_pymnt_amnt,0.038940
total_rec_int,0.035240
total_pymnt,0.033699
max_bal_bc,0.027801
revol_bal,0.027471
total_acc,0.026448
loan_amnt,0.026009
inq_fi,0.021718
tot_cur_bal,0.021049


### Easy Ensemble AdaBoost Classifier

In [18]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

ee_model = EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
ee_model.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, n_jobs=1, random_state=1)

In [19]:
# Calculated the balanced accuracy score
predictions = ee_model.predict(X_test_scaled)

balanced_accuracy_score(y_test, predictions)

0.5572294191391559

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    matrix, index=["Actual High-Risk", "Actual Low-Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm_df

,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,3,7
Actual Low-Risk,444,1949


In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.30      0.81      0.01      0.49      0.23        10
   low_risk       1.00      0.81      0.30      0.90      0.49      0.26      2393

avg / total       0.99      0.81      0.30      0.89      0.49      0.26      2403

